In [66]:
import json 
with open('example_3.json', 'r') as f:
    data = json.load(f)


def get_color(r,g,b):
    if r==0.5 and g==0.5 and b==0:
        return 'olive'
    if r==0.5 and g==0 and b==0.5:
        return 'purple'
    if r==0 and g==0.5 and b==0.5:
        return 'teal'
    
def get_m(m):
    return m.split('_')[0]


static = data['initial_static_objects']
shape = []
color = []
material = []
for i in range(len(static)):
    shape.append(static[i]['name'])
    c = static[i]['color']
    r = c['r']
    g = c['g']
    b = c['b']
    c2 = get_color(r,g,b)
    color.append(c2)
    m=static[i]['material']
    m2 = get_m(m)
    material.append(m2)

static_ob = []
s=''
for i in range(len(static)):
    s= color[i]+' '+material[i]+' '+shape[i]
    static_ob.append(s)

dynamic = data["iniital_dynamic_objects"]
shape2 = dynamic[0]['name']
c = dynamic[0]['color']
r = c['r']
g = c['g']
b = c['b']
c2 = get_color(r,g,b)
color2 = c2
m=static[i]['material']
m2 = get_m(m)
material2 = m2
dynamic_ob=color2 +' '+material2+' '+shape2

col = data['collisions']
noc=0
for i in range(len(col)):
    state = col[i]['state']
    if state == 'enter':
        noc+=1

#noc = str(noc)
static_ob.append(dynamic_ob)
static_ob.append(noc)
#static objects
print(static_ob)


['purple cardboard cube', 'teal aluminium sphere', 'olive cardboard sphere', 'teal aluminium cube', 'purple aluminium cube', 4]


In [67]:
import csv
import pandas as pd
import math
import numpy as np
import os
import json
import warnings
warnings.filterwarnings("ignore")

def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def num(v):
    if v == 1:
        return 'one'
    elif v == 2:
        return 'two'
    elif v == 3:
        return 'three'
    elif v == 4:
        return 'four'
    elif v == 5:
        return 'five'
    elif v == 6:
        return 'six'
    elif v == 7:
        return 'seven'
    elif v == 8:
        return 'eight'
    elif v == 9:
        return 'nine'
    

def get_direction_origin(x_a, y_a):
    direction = ""

    if x_a > 0 and y_a > 0:
        direction = "northeast"
    elif x_a > 0 and y_a < 0:
        direction = "southeast"
    elif x_a < 0 and y_a < 0:
        direction = "southwest"
    elif x_a < 0 and y_a > 0:
        direction = "northwest"
    elif x_a > 0:
        direction = "east"
    elif y_a > 0:
        direction = "north"
    elif x_a < 0:
        direction = "west"
    elif y_a < 0:
        direction = "south"

    return direction

def get_direction(x_a, y_a, x_b, y_b):
    x_diff = x_b - x_a
    y_diff = y_b - y_a
    directions = ""

    angle = math.degrees(math.atan2(y_diff, x_diff))  # Convert negative angles to positive
    if angle<0:
        angle = angle+360

    if angle>337.5 and angle<=22.5 and angle==0 and angle==360:
        directions = 'East'
    elif angle>22.5 and angle<=67.5:
        directions = 'Northeast'
    elif angle>67.5 and angle<=112.5:
        directions = 'North'
    elif angle>112.5 and angle<=157.5:
        directions = 'Northwest'
    elif angle>157.5 and angle<=202.5:
        directions = 'West'
    elif angle>202.5 and angle<=247.5:
        directions = 'Southwest'
    elif angle>247.5 and angle<=292.5:
        directions= 'South'
    elif angle>292.5 and angle<=337.5:
        directions = 'Southeast'

    return directions

def get_direction2(x_a, y_a, x_b, y_b):
    x_diff = x_b - x_a
    y_diff = y_b - y_a
    directions = ""

    angle = math.degrees(math.atan2(y_diff, x_diff))  # Convert negative angles to positive
    if angle<0:
        angle = angle+360

    if angle>337.5 and angle<=22.5 and angle==0 and angle==360:
        directions = 'West'
    elif angle>22.5 and angle<=67.5:
        directions = 'Southwest'
    elif angle>67.5 and angle<=112.5:
        directions = 'South'
    elif angle>112.5 and angle<=157.5:
        directions = 'Southeast'
    elif angle>157.5 and angle<=202.5:
        directions = 'East'
    elif angle>202.5 and angle<=247.5:
        directions = 'Northeast'
    elif angle>247.5 and angle<=292.5:
        directions= 'North'
    elif angle>292.5 and angle<=337.5:
        directions = 'Northwest'

    return directions

def is_colliding(box1, box2):
    x1_box1 = int(box1['x1'].iloc[0])
    y1_box1 = int(box1['y1'].iloc[0])
    x2_box1 = int(box1['x2'].iloc[0])
    y2_box1 = int(box1['y2'].iloc[0])

    x1_box2 = int(box2['x1'])
    y1_box2 = int(box2['y1'])
    x2_box2 = int(box2['x2'])
    y2_box2 = int(box2['y2'])

    centerx_box1 = int((x1_box1 + x2_box1)/2)
    centery_box1 = int((y1_box1 + y2_box1)/2)

    centerx_box2 = int((x1_box2 + x2_box2)/2)
    centery_box2 = int((y1_box2 + y2_box2)/2)

    center_distance = math.sqrt((centerx_box2 - centerx_box1)**2 + (centery_box2 - centery_box1)**2)

    half_width_sum = ((x2_box1 - x1_box1) + (x2_box2 - x1_box2))/2
    half_height_sum = ((y1_box1 - y2_box1) + (y1_box2 - y2_box2))/2

    #print(center_distance, half_width_sum, half_height_sum)

    if half_width_sum >= center_distance:
        return True

    if half_height_sum >= center_distance:
        return True

    else :
        return False

df = pd.read_csv(f'out_csv/example_3.csv')
y_static_ob = []
class_order = df[df['frames'] == 'frame0']['class'].tolist()
df = df[df['class'].isin(class_order)].copy()
df['y1'] = -df['y1']
df['y2'] = -df['y2']
df['x1'] -= 256
df['y1'] += 256
df['x2'] -= 256
df['y2'] += 256

con = df['frames'] == 'frame0'
class_id = df.loc[con, 'class']
nob = num(len(list(class_id)))

s1=''
#s1 = f'Initially there are {nob} objects, {num(len(list(class_id))-1)} static and one dynamic object. '
s1 = f'Initially there are {nob} objects, {num(len(list(class_id))-1)} static and one dynamic object. The static objects are '

frame_0 = df[df['frames'] == 'frame0']
frame_0['center'] = frame_0.apply(lambda row: ((row['x1'] + row['x2']) / 2, (row['y1'] + row['y2']) / 2), axis=1)

s2=''
for index, row in frame_0.iterrows():
    current_center = row['center']
    df_without_current = frame_0[frame_0.index != index]  
    current_object = row['class']
    x1_0 = row['x1']
    y1_0 = row['y1']
    d = get_direction_origin(x1_0, y1_0)
    
    nearest_distance = float('inf')
    nearest_object = None
    nearest_center = None
    
    for _, other_row in df_without_current.iterrows():
        other_center = other_row['center']
        distance = calculate_distance(current_center, other_center)
        
        if distance < nearest_distance:
            nearest_distance = distance
            nearest_object = other_row['class']
            nearest_center = other_center
            x1_1 = other_row['x1']
            y1_1 = other_row['y1']

    pos = get_direction2(x1_0, y1_0, x1_1, y1_1)

    #s2 = s2+ f'The {current_object} is located in the {d} direction from the origin and {pos} of {nearest_object}. '
    s2 = s2 + f'{current_object}, '
    y_static_ob.append(current_object)

frame_0 = df[df['frames'] == 'frame0']
class_present = list(frame_0['class'])
changes_det = False
dynamic=[]
for i in range(len(class_present)):
    frame_p = df[df['class'] == class_present[i]]
    frame_p = frame_p.reset_index(drop=True)
    frame_p1 = frame_p.drop(columns=['frames', 'class', 'x2', 'y2'])
    frame_p2 = frame_p.drop(columns=['frames', 'class', 'x1', 'y1'])
    for j in range(1, len(frame_p)):
        prev_frame1 = frame_p1.iloc[j-1]
        curr_frame1 = frame_p1.iloc[j]

        prev_frame2 = frame_p2.iloc[j-1]
        curr_frame2 = frame_p2.iloc[j]

        if not prev_frame1.equals(curr_frame1) and not prev_frame2.equals(curr_frame2):
            changes_det = True
            first_change = j
            break
    
    if changes_det == True:
        dynamic.append((class_present[i], frame_p.iloc[first_change,0], j))

    changes_det = False


sorted_dynamic = sorted(dynamic, key=lambda x:x[2])
dynamic1 = sorted_dynamic[0]
sorted_dynamic = sorted_dynamic[1:]

for i in range(len(sorted_dynamic)):
    frame = sorted_dynamic[i][1]
    ob = sorted_dynamic[i][0]
    frame_n = df[df['frames'] == frame]
    frame_n = frame_n.reset_index(drop=True)
    row = frame_n[frame_n['class'] == ob]
    bbox_1 = row[['x1', 'y1', 'x2', 'y2']]
    ind = bbox_1.index
    ind = ind[0]
    for j in range(len(frame_n)):
        if j!=ind:
            row = frame_n.iloc[j]
            bbox_n = row[['x1', 'y1', 'x2', 'y2']]
            res = is_colliding(bbox_1, bbox_n)
            if res == True:
                cl = list(row[['class']])
                sorted_dynamic[i] = sorted_dynamic[i]+(cl[0],)


y_dynamic_ob = dynamic1[0]
y_static_ob.append(y_dynamic_ob)
y_noc = len(sorted_dynamic)
y_static_ob.append(y_noc)
print(y_static_ob)

['teal aluminium cube', 'olive cardboard sphere', 'purple cardboard cube', 'purple aluminium cube', 'teal aluminium sphere', 'purple aluminium cube', 4]


tp = ache and positive
fp = nei but identifies
fn = ache but cannot identify

In [69]:
a=[4]
if isinstance(a[0], int):
    print(True)

True


In [72]:
pac = {"tp":0, "fp":0, "fn":0}
pas = {"tp":0, "fp":0, "fn":0}
pcs = {"tp":0, "fp":0, "fn":0}
pcc = {"tp":0, "fp":0, "fn":0}
tac = {"tp":0, "fp":0, "fn":0}
tas = {"tp":0, "fp":0, "fn":0}
tcc = {"tp":0, "fp":0, "fn":0}
tcs = {"tp":0, "fp":0, "fn":0}
oac = {"tp":0, "fp":0, "fn":0}
oas = {"tp":0, "fp":0, "fn":0}
occ = {"tp":0, "fp":0, "fn":0}
ocs = {"tp":0, "fp":0, "fn":0}
colx = {"tp":0, "fp":0, "fn":0}

def update(item, x):
    if item == 'purple aluminium cube':
        pac[x]+=1
    if item == 'purple aluminium sphere':
        pas[x]+=1   
    if item == 'purple cardboard sphere':
        pcs[x]+=1
    if item == 'purple cardboard cube':
        pcc[x]+=1 
    if item == 'teal aluminium cube':
        tac[x]+=1
    if item == 'teal aluminium sphere':
        tas[x]+=1   
    if item == 'teal cardboard cube':
        tcc[x]+=1
    if item == 'teal cardboard sphere':
        tcs[x]+=1 
    if item == 'olive aluminium cube':
        oac[x]+=1
    if item == 'olive aluminium sphere':
        oas[x]+=1   
    if item == 'olive cardboard cube':
        occ[x]+=1
    if item == 'olive cardboard sphere':
        ocs[x]+=1 
    if isinstance(item, int):
        colx[x]+=1

def display():
    print(f"purple aluminium cube: {pac}")
    print(f"purple aluminium sphere: {pas}")
    print(f"purple cardboard cube: {pcc}")
    print(f"purple cardboard sphere: {pcs}")
    print(f"teal aluminium cube: {tac}")
    print(f"teal aluminium sphere: {tas}")
    print(f"teal cardboard cube: {tcc}")
    print(f"teal cardboard sphere: {tcs}")
    print(f"olive aluminium cube: {oac}")
    print(f"olive aluminium sphere: {oas}")
    print(f"olive cardboard cube: {occ}")
    print(f"olive cardboard sphere: {ocs}")
    print(f"noc: {colx}")

for item in y_static_ob:
    if item in static_ob:
        #tp
        update(item, "tp")
        #print(item)
    if item not in static_ob:
        #fp
        update(item, "fp")
        #print(item)

for item in static_ob:
    if item not in y_static_ob:
        #fn
        update(item, "fn")
        #print(item)

display()
print(static_ob)
print(y_static_ob)



purple aluminium cube: {'tp': 2, 'fp': 0, 'fn': 0}
purple aluminium sphere: {'tp': 0, 'fp': 0, 'fn': 0}
purple cardboard cube: {'tp': 1, 'fp': 0, 'fn': 0}
purple cardboard sphere: {'tp': 0, 'fp': 0, 'fn': 0}
teal aluminium cube: {'tp': 1, 'fp': 0, 'fn': 0}
teal aluminium sphere: {'tp': 1, 'fp': 0, 'fn': 0}
teal cardboard cube: {'tp': 0, 'fp': 0, 'fn': 0}
teal cardboard sphere: {'tp': 0, 'fp': 0, 'fn': 0}
olive aluminium cube: {'tp': 0, 'fp': 0, 'fn': 0}
olive aluminium sphere: {'tp': 0, 'fp': 0, 'fn': 0}
olive cardboard cube: {'tp': 0, 'fp': 0, 'fn': 0}
olive cardboard sphere: {'tp': 1, 'fp': 0, 'fn': 0}
noc: {'tp': 1, 'fp': 0, 'fn': 0}
['purple cardboard cube', 'teal aluminium sphere', 'olive cardboard sphere', 'teal aluminium cube', 'purple aluminium cube', 4]
['teal aluminium cube', 'olive cardboard sphere', 'purple cardboard cube', 'purple aluminium cube', 'teal aluminium sphere', 'purple aluminium cube', 4]


purple aluminium cube  
purple aluminium sphere
teal aluminium cube
teal aluminium sphere
teal cardboard cube
teal cardboard sphere
olive aluminium cube
olive aluminium sphere
olive cardboard cube
olive cardboard sphere
purple cardboard sphere
purple cardboard cube

In [2]:
import json
import os 
import csv
import pandas as pd
import math
import numpy as np
import warnings
warnings.filterwarnings("ignore")


pac = {"tp":0, "fp":0, "fn":0}
pas = {"tp":0, "fp":0, "fn":0}
pcs = {"tp":0, "fp":0, "fn":0}
pcc = {"tp":0, "fp":0, "fn":0}
tac = {"tp":0, "fp":0, "fn":0}
tas = {"tp":0, "fp":0, "fn":0}
tcc = {"tp":0, "fp":0, "fn":0}
tcs = {"tp":0, "fp":0, "fn":0}
oac = {"tp":0, "fp":0, "fn":0}
oas = {"tp":0, "fp":0, "fn":0}
occ = {"tp":0, "fp":0, "fn":0}
ocs = {"tp":0, "fp":0, "fn":0}
colx = {"tp":0, "fp":0, "fn":0}

def display():
    print(f"purple aluminium cube: {pac}")
    print(f"purple aluminium sphere: {pas}")
    print(f"purple cardboard cube: {pcc}")
    print(f"purple cardboard sphere: {pcs}")
    print(f"teal aluminium cube: {tac}")
    print(f"teal aluminium sphere: {tas}")
    print(f"teal cardboard cube: {tcc}")
    print(f"teal cardboard sphere: {tcs}")
    print(f"olive aluminium cube: {oac}")
    print(f"olive aluminium sphere: {oas}")
    print(f"olive cardboard cube: {occ}")
    print(f"olive cardboard sphere: {ocs}")
    print(f"noc: {colx}")

def update(item, x):
    if item == 'purple aluminium cube':
        pac[x]+=1
    if item == 'purple aluminium sphere':
        pas[x]+=1   
    if item == 'purple cardboard sphere':
        pcs[x]+=1
    if item == 'purple cardboard cube':
        pcc[x]+=1 
    if item == 'teal aluminium cube':
        tac[x]+=1
    if item == 'teal aluminium sphere':
        tas[x]+=1   
    if item == 'teal cardboard cube':
        tcc[x]+=1
    if item == 'teal cardboard sphere':
        tcs[x]+=1 
    if item == 'olive aluminium cube':
        oac[x]+=1
    if item == 'olive aluminium sphere':
        oas[x]+=1   
    if item == 'olive cardboard cube':
        occ[x]+=1
    if item == 'olive cardboard sphere':
        ocs[x]+=1 
    if isinstance(item, int):
        colx[x]+=1

def calculate_distance(p1, p2):
    return np.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

def num(v):
    if v == 1:
        return 'one'
    elif v == 2:
        return 'two'
    elif v == 3:
        return 'three'
    elif v == 4:
        return 'four'
    elif v == 5:
        return 'five'
    elif v == 6:
        return 'six'
    elif v == 7:
        return 'seven'
    elif v == 8:
        return 'eight'
    elif v == 9:
        return 'nine'
    

def get_direction_origin(x_a, y_a):
    direction = ""

    if x_a > 0 and y_a > 0:
        direction = "northeast"
    elif x_a > 0 and y_a < 0:
        direction = "southeast"
    elif x_a < 0 and y_a < 0:
        direction = "southwest"
    elif x_a < 0 and y_a > 0:
        direction = "northwest"
    elif x_a > 0:
        direction = "east"
    elif y_a > 0:
        direction = "north"
    elif x_a < 0:
        direction = "west"
    elif y_a < 0:
        direction = "south"

    return direction

def get_direction(x_a, y_a, x_b, y_b):
    x_diff = x_b - x_a
    y_diff = y_b - y_a
    directions = ""

    angle = math.degrees(math.atan2(y_diff, x_diff))  # Convert negative angles to positive
    if angle<0:
        angle = angle+360

    if angle>337.5 and angle<=22.5 and angle==0 and angle==360:
        directions = 'East'
    elif angle>22.5 and angle<=67.5:
        directions = 'Northeast'
    elif angle>67.5 and angle<=112.5:
        directions = 'North'
    elif angle>112.5 and angle<=157.5:
        directions = 'Northwest'
    elif angle>157.5 and angle<=202.5:
        directions = 'West'
    elif angle>202.5 and angle<=247.5:
        directions = 'Southwest'
    elif angle>247.5 and angle<=292.5:
        directions= 'South'
    elif angle>292.5 and angle<=337.5:
        directions = 'Southeast'

    return directions

def get_direction2(x_a, y_a, x_b, y_b):
    x_diff = x_b - x_a
    y_diff = y_b - y_a
    directions = ""

    angle = math.degrees(math.atan2(y_diff, x_diff))  # Convert negative angles to positive
    if angle<0:
        angle = angle+360

    if angle>337.5 and angle<=22.5 and angle==0 and angle==360:
        directions = 'West'
    elif angle>22.5 and angle<=67.5:
        directions = 'Southwest'
    elif angle>67.5 and angle<=112.5:
        directions = 'South'
    elif angle>112.5 and angle<=157.5:
        directions = 'Southeast'
    elif angle>157.5 and angle<=202.5:
        directions = 'East'
    elif angle>202.5 and angle<=247.5:
        directions = 'Northeast'
    elif angle>247.5 and angle<=292.5:
        directions= 'North'
    elif angle>292.5 and angle<=337.5:
        directions = 'Northwest'

    return directions

def is_colliding(box1, box2):
    x1_box1 = int(box1['x1'].iloc[0])
    y1_box1 = int(box1['y1'].iloc[0])
    x2_box1 = int(box1['x2'].iloc[0])
    y2_box1 = int(box1['y2'].iloc[0])

    x1_box2 = int(box2['x1'])
    y1_box2 = int(box2['y1'])
    x2_box2 = int(box2['x2'])
    y2_box2 = int(box2['y2'])

    centerx_box1 = int((x1_box1 + x2_box1)/2)
    centery_box1 = int((y1_box1 + y2_box1)/2)

    centerx_box2 = int((x1_box2 + x2_box2)/2)
    centery_box2 = int((y1_box2 + y2_box2)/2)

    center_distance = math.sqrt((centerx_box2 - centerx_box1)**2 + (centery_box2 - centery_box1)**2)

    half_width_sum = ((x2_box1 - x1_box1) + (x2_box2 - x1_box2))/2
    half_height_sum = ((y1_box1 - y2_box1) + (y1_box2 - y2_box2))/2

    #print(center_distance, half_width_sum, half_height_sum)

    if half_width_sum >= center_distance:
        return True

    if half_height_sum >= center_distance:
        return True

    else :
        return False
    
def get_color(r,g,b):
    if r==0.5 and g==0.5 and b==0:
        return 'olive'
    if r==0.5 and g==0 and b==0.5:
        return 'purple'
    if r==0 and g==0.5 and b==0.5:
        return 'teal'
    
def get_m(m):
    return m.split('_')[0]

path = 'test'
count = 0
#########################################################################
for i in range(5100):
    try:
        csv_file = f'csv/example_{i}.csv'
        json_file = f'json/example_{i}.json'
        csv_path = os.path.join(path, csv_file)
        json_path = os.path.join(path, json_file)

        if not os.path.exists(csv_path):
            continue
        if not os.path.exists(json_path):
            continue

        count+=1

        with open(json_path, 'r') as f:
            data = json.load(f)

        static = data['initial_static_objects']
        shape = []
        color = []
        material = []
        for i in range(len(static)):
            shape.append(static[i]['name'])
            c = static[i]['color']
            r = c['r']
            g = c['g']
            b = c['b']
            c2 = get_color(r,g,b)
            color.append(c2)
            m=static[i]['material']
            m2 = get_m(m)
            material.append(m2)

        static_ob = []
        s=''
        for i in range(len(static)):
            s= color[i]+' '+material[i]+' '+shape[i]
            static_ob.append(s)

        dynamic = data["iniital_dynamic_objects"]
        shape2 = dynamic[0]['name']
        c = dynamic[0]['color']
        r = c['r']
        g = c['g']
        b = c['b']
        c2 = get_color(r,g,b)
        color2 = c2
        m=static[i]['material']
        m2 = get_m(m)
        material2 = m2
        dynamic_ob=color2 +' '+material2+' '+shape2

        col = data['collisions']
        noc=0
        for i in range(len(col)):
            state = col[i]['state']
            if state == 'enter':
                noc+=1

        #noc = str(noc)
        static_ob.append(dynamic_ob)
        static_ob.append(noc)
        #static objects
        #print(static_ob)
        

        

        df = pd.read_csv(csv_path)
        y_static_ob = []
        class_order = df[df['frames'] == 'frame0']['class'].tolist()
        df = df[df['class'].isin(class_order)].copy()
        df['y1'] = -df['y1']
        df['y2'] = -df['y2']
        df['x1'] -= 256
        df['y1'] += 256
        df['x2'] -= 256
        df['y2'] += 256

        con = df['frames'] == 'frame0'
        class_id = df.loc[con, 'class']
        nob = num(len(list(class_id)))

        s1=''
        #s1 = f'Initially there are {nob} objects, {num(len(list(class_id))-1)} static and one dynamic object. '
        s1 = f'Initially there are {nob} objects, {num(len(list(class_id))-1)} static and one dynamic object. The static objects are '

        frame_0 = df[df['frames'] == 'frame0']
        frame_0['center'] = frame_0.apply(lambda row: ((row['x1'] + row['x2']) / 2, (row['y1'] + row['y2']) / 2), axis=1)

        s2=''
        for index, row in frame_0.iterrows():
            current_center = row['center']
            df_without_current = frame_0[frame_0.index != index]  
            current_object = row['class']
            x1_0 = row['x1']
            y1_0 = row['y1']
            d = get_direction_origin(x1_0, y1_0)
            
            nearest_distance = float('inf')
            nearest_object = None
            nearest_center = None
            
            for _, other_row in df_without_current.iterrows():
                other_center = other_row['center']
                distance = calculate_distance(current_center, other_center)
                
                if distance < nearest_distance:
                    nearest_distance = distance
                    nearest_object = other_row['class']
                    nearest_center = other_center
                    x1_1 = other_row['x1']
                    y1_1 = other_row['y1']

            pos = get_direction2(x1_0, y1_0, x1_1, y1_1)

            #s2 = s2+ f'The {current_object} is located in the {d} direction from the origin and {pos} of {nearest_object}. '
            s2 = s2 + f'{current_object}, '
            y_static_ob.append(current_object)

        frame_0 = df[df['frames'] == 'frame0']
        class_present = list(frame_0['class'])
        changes_det = False
        dynamic=[]
        for i in range(len(class_present)):
            frame_p = df[df['class'] == class_present[i]]
            frame_p = frame_p.reset_index(drop=True)
            frame_p1 = frame_p.drop(columns=['frames', 'class', 'x2', 'y2'])
            frame_p2 = frame_p.drop(columns=['frames', 'class', 'x1', 'y1'])
            for j in range(1, len(frame_p)):
                prev_frame1 = frame_p1.iloc[j-1]
                curr_frame1 = frame_p1.iloc[j]

                prev_frame2 = frame_p2.iloc[j-1]
                curr_frame2 = frame_p2.iloc[j]

                if not prev_frame1.equals(curr_frame1) and not prev_frame2.equals(curr_frame2):
                    changes_det = True
                    first_change = j
                    break
            
            if changes_det == True:
                dynamic.append((class_present[i], frame_p.iloc[first_change,0], j))

            changes_det = False


        sorted_dynamic = sorted(dynamic, key=lambda x:x[2])
        dynamic1 = sorted_dynamic[0]
        sorted_dynamic = sorted_dynamic[1:]

        for i in range(len(sorted_dynamic)):
            frame = sorted_dynamic[i][1]
            ob = sorted_dynamic[i][0]
            frame_n = df[df['frames'] == frame]
            frame_n = frame_n.reset_index(drop=True)
            row = frame_n[frame_n['class'] == ob]
            bbox_1 = row[['x1', 'y1', 'x2', 'y2']]
            ind = bbox_1.index
            ind = ind[0]
            for j in range(len(frame_n)):
                if j!=ind:
                    row = frame_n.iloc[j]
                    bbox_n = row[['x1', 'y1', 'x2', 'y2']]
                    res = is_colliding(bbox_1, bbox_n)
                    if res == True:
                        cl = list(row[['class']])
                        sorted_dynamic[i] = sorted_dynamic[i]+(cl[0],)


        y_dynamic_ob = dynamic1[0]
        y_static_ob.append(y_dynamic_ob)
        y_noc = len(sorted_dynamic)
        y_static_ob.append(y_noc)
        #print(y_static_ob)

        for item in y_static_ob:
            if item in static_ob:
                #tp
                update(item, "tp")
                #print(item)
            if item not in static_ob:
                #fp
                update(item, "fp")
                #print(item)

        for item in static_ob:
            if item not in y_static_ob:
                #fn
                update(item, "fn")
                #print(item)
        
    except: pass
display()

sum_tp = pac["tp"]+pas["tp"]+pcc["tp"]+pcs["tp"]+tac["tp"]+tas["tp"]+tcc["tp"]+tcs["tp"]+oac["tp"]+oas["tp"]+occ["tp"]+ocs["tp"]+colx["tp"]
sum_fp = pac["fp"]+pas["fp"]+pcc["fp"]+pcs["fp"]+tac["fp"]+tas["fp"]+tcc["fp"]+tcs["fp"]+oac["fp"]+oas["fp"]+occ["fp"]+ocs["fp"]+colx["fp"]
sum_fn = pac["fn"]+pas["fn"]+pcc["fn"]+pcs["fn"]+tac["fn"]+tas["fn"]+tcc["fn"]+tcs["fn"]+oac["fn"]+oas["fn"]+occ["fn"]+ocs["fn"]+colx["fn"]

micro_pre = sum_tp/(sum_tp+sum_fp)
micro_rec = sum_tp/(sum_tp+sum_fn)
micro_f1 = 2*(micro_pre*micro_rec)/(micro_pre+micro_rec)

print(f"files evaluated: {count}")
print(f"precision: {micro_pre}" )
print(f"recall: {micro_rec}")
print(f"micro f1: {micro_f1}")


purple aluminium cube: {'tp': 1948, 'fp': 353, 'fn': 129}
purple aluminium sphere: {'tp': 2148, 'fp': 465, 'fn': 79}
purple cardboard cube: {'tp': 2131, 'fp': 471, 'fn': 82}
purple cardboard sphere: {'tp': 1857, 'fp': 293, 'fn': 117}
teal aluminium cube: {'tp': 2109, 'fp': 397, 'fn': 114}
teal aluminium sphere: {'tp': 1858, 'fp': 289, 'fn': 107}
teal cardboard cube: {'tp': 1936, 'fp': 426, 'fn': 112}
teal cardboard sphere: {'tp': 1891, 'fp': 332, 'fn': 78}
olive aluminium cube: {'tp': 2134, 'fp': 424, 'fn': 107}
olive aluminium sphere: {'tp': 2129, 'fp': 459, 'fn': 104}
olive cardboard cube: {'tp': 1886, 'fp': 376, 'fn': 135}
olive cardboard sphere: {'tp': 2076, 'fp': 441, 'fn': 113}
noc: {'tp': 1633, 'fp': 2750, 'fn': 2750}
files evaluated: 4385
precision: 0.7749006383234975
recall: 0.8646977791217283
micro f1: 0.817340214370782
